In [1]:
import numpy as np
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import clone_model
from models import get_model, resnet_v1, resnet_v2
from util import select_clean_uncertain, combine_result, inject_noise
import time
import argparse
from util import other_class
from tensorflow.python.lib.io import file_io
from keras.utils import np_utils, multi_gpu_model
from keras import backend as K
from loss_acc_plot import loss_acc_plot
from sklearn.metrics import accuracy_score
# from io import BytesIO

In [2]:
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [3]:
NUM_CLASSES = {'mnist': 10, 'svhn': 10, 'cifar-10': 10, 'cifar-100': 100, 'celeb': 20}
dataset = "celeb"
noise_ratio = 0
data_ratio = 100
# X_train, y_train, X_test, y_test, un_selected_index = get_data('celeb', init_noise_ratio, data_ratio, random_shuffle=False)

In [4]:
def eval_model(model, test_data, test_labels):
    predictions = model.predict(test_data)
    #print(predictions)
    #predictions = list(np.around(np.array(predictions),0))
    predictions_ = np.argmax(predictions, axis = 1)
    predictions_test = np.argmax(test_labels, axis = 1)
    
    accuracy = accuracy_score(predictions_test, predictions_)
    #print(accuracy)
    return accuracy

In [5]:
def get_data(noise_ratio):
    X_train = np.load('data/image_train_20.npy')
    X_test = np.load('data/image_test_20.npy')
    y_train = np.load('data/label_train_20.npy')
    y_test = np.load('data/label_test_20.npy')

    image_shape = 128
    X_train = X_train.reshape(-1, image_shape, image_shape, 3)
    X_test = X_test.reshape(-1, image_shape, image_shape, 3)

    X_train = X_train / 255.0
    X_test = X_test / 255.0

    means = X_train.mean(axis=0)
    # std = np.std(X_train)
    X_train = (X_train - means)  # / std
    X_test = (X_test - means)  # / std

    # they are 2D originally in cifar
    y_train = y_train.ravel()
    y_test = y_test.ravel()
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')

    if noise_ratio > 0:
            n_samples = y_train.shape[0]
            n_noisy = int(noise_ratio*n_samples/100)
            noisy_idx = np.random.choice(n_samples, n_noisy, replace=False)
            for i in noisy_idx:
                y_train[i] = other_class(n_classes=NUM_CLASSES[dataset], current_class=y_train[i])
    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    y_train = np_utils.to_categorical(y_train, NUM_CLASSES[dataset])
    y_test = np_utils.to_categorical(y_test, NUM_CLASSES[dataset])
    return X_train, X_test, y_train, y_test

In [6]:
from sklearn.neural_network import MLPClassifier

In [7]:
# MLP thermostat
mlpmodel = MLPClassifier(solver='adam', hidden_layer_sizes=(28,28), random_state=1)
acc_list_mlp = []
repeat = 10
for i in range(repeat):
    for noise_ratio in [0,10,20,30,40,50,60,70,80,90,100]:
        X_train, X_test, y_train, y_test = get_data(noise_ratio)  
        mlpmodel.fit(X_train.reshape(X_train.shape[0],-1), y_train)
        acc = eval_model(mlpmodel, X_test.reshape(X_test.shape[0], -1), y_test)
        acc_list_mlp.append(acc)
        print("noise_level, accuracy: ", noise_ratio, acc)

(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  0 0.5488721804511278
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  10 0.4842105263157895
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  20 0.37293233082706767
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  30 0.3022556390977444
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  40 0.2586466165413534
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  50 0.22255639097744362
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  60 0.1593984962406015
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  70 0.11578947368421053
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  80 0.07669172932330827
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  90 0.05864661654135

/home/zilong/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


noise_level, accuracy:  10 0.4706766917293233
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  20 0.35037593984962406
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  30 0.35037593984962406
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  40 0.2781954887218045
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  50 0.20150375939849624
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  60 0.1699248120300752
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  70 0.1293233082706767
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  80 0.11578947368421053
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  90 0.06315789473684211
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  100 0.05714285714285714
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) 

/home/zilong/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


noise_level, accuracy:  10 0.49624060150375937
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  20 0.42105263157894735
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  30 0.34135338345864663
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  40 0.27518796992481204
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  50 0.24210526315789474
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  60 0.1518796992481203
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  70 0.12330827067669173
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  80 0.08421052631578947
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  90 0.06917293233082707
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
noise_level, accuracy:  100 0.05413533834586466
(2639, 128, 128, 3) (665, 128, 128, 3) (2639

In [8]:
avg_result = np.zeros(11)
for i in range(repeat):
    for j in range(11):
        avg_result[j] += acc_list_mlp[j+i*11]
average = avg_result/repeat
print(average)

[0.54887218 0.4724812  0.38390977 0.33067669 0.26135338 0.20706767
 0.1687218  0.12691729 0.09443609 0.06796992 0.04977444]
